In [1]:
import pandas as pd
import json
import pickle
import os
import itertools
from ast import literal_eval
import numpy as np
import sys
from datetime import datetime
from scipy import stats
from IPython.display import clear_output, display
import subprocess
from datetime import datetime
pd.options.display.float_format = '{:.0f}'.format

In [2]:
import pandas as pd
import numpy as np
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
french_stopwords = list(fr_stop)
from scipy import stats
import math
import string
import igraph as ig

In [3]:
sys.path.append("/home/osboxes/proj/streamer/")
from fun import *

print("")

In [4]:
#Functions

In [5]:
def BuildGraphFrom2DF(FinalInf,graphdf):

    """
    Input : un dataframe des nodes
    Input : un dataframe des links
    Output : un objet graph igraph
    """
    
    g = ig.Graph(directed=False)
    nodesids = FinalInf.GRAPHID.values
    g.add_vertices(nodesids)

    g.vs["AUTHORDESCRIPTION"] = FinalInf.AUTHORDESCRIPTION.tolist()
    g.vs["AUTHORFNAME"] = FinalInf.AUTHORFNAME.tolist()
    g.vs["AUTHORID"] = FinalInf.AUTHORID.tolist()
    g.vs["AUTHORNAME"] = FinalInf.AUTHORNAME.tolist()
    g.vs["AUTHORFOLLOWERS"] = FinalInf.AUTHORFOLLOWERS.tolist()

    edgeslist = graphdf[["GRAPHIDA","GRAPHIDB"]].values.tolist()
    g.add_edges(edgeslist)
    g.es["weight"] = graphdf.f.tolist()

    return g


def DeleteSomeNodes(g,AttributeName,ThresholdFollowersInf,ThresholdFollowersSup):

    """
    Input : un objet igraph graph
    Input : borne inferieur et superieur
    Input : Attribute Name
    Output : un objet igraph graph
    """
    
    
    to_delete_ids = [node for node in g.vs if node[AttributeName]<ThresholdFollowersInf]
    g.delete_vertices(to_delete_ids)

    to_delete_ids = [node for node in g.vs if node[AttributeName]>ThresholdFollowersSup]
    g.delete_vertices(to_delete_ids)

    return g

In [6]:
def DefineFirstTm(x):

    x.sort(key = lambda a : a[1], reverse = False)
    if len(x)>0:
        firsttm = x[0][1]
    else:
        firsttm = None

    return firsttm

def ExtractNextTuple(firsttm,x,tolerance = 100):
    
    if firsttm is None:
        return None    
    
    x.sort(key = lambda a : a[1], reverse = False)
    ecart = np.abs(firsttm - np.array([item[0] for item in x]))
    idinteresting = np.argmin(ecart)
    
    if idinteresting+1==len(x):
        return None
    
    if ecart[idinteresting]>tolerance:
        solution = None
    else:
        solution = x[idinteresting]

    return solution


def ExtractCompletePeriods(x,tolerancevalue):
    
    """
    Input : liste de tuples représentant les bornes inf et sup de périodes
    Output : Liste de périodes complètes
    """

    firsttm = DefineFirstTm(x)
    L = []
    NextTuple = 0

    while NextTuple is not None:
        NextTuple = ExtractNextTuple(firsttm,x,tolerancevalue)
        if NextTuple is not None:
            firsttm = NextTuple[1]
            L.append(NextTuple)

    return L

In [9]:
#Load and clean

In [507]:
graphdf = pd.read_csv("graph1581156942.csv")
graphdf.sort_values(by="f",ascending=False,inplace=True)
nodesids = np.unique(np.hstack((graphdf.a.values,graphdf.b.values)))

FinalInf = PickleLoad("FinalInf.pkl")
FinalInf = FinalInf[FinalInf.AUTHORID.isin(nodesids)]
FinalInf = FinalInf.reset_index(drop=True).reset_index()
FinalInf.rename(columns = {"index":"GRAPHID"},inplace=True)

graphdf = graphdf.merge(FinalInf[["GRAPHID","AUTHORID"]],how="left",left_on="a",right_on="AUTHORID").rename(columns={"GRAPHID":"GRAPHIDA"})
graphdf = graphdf.merge(FinalInf[["GRAPHID","AUTHORID"]],how="left",left_on="b",right_on="AUTHORID").rename(columns={"GRAPHID":"GRAPHIDB"})
graphdf.drop(columns=["AUTHORID_x","AUTHORID_y"],inplace = True)

In [508]:
#Graph

In [544]:
ThresholdFollowersInf = 20000
ThresholdFollowersSup = 80000
AttributeName = "AUTHORFOLLOWERS"
g = BuildGraphFrom2DF(FinalInf,graphdf)
g = DeleteSomeNodes(g,AttributeName,ThresholdFollowersInf,ThresholdFollowersSup)
todelete = [node for node in g.vs if node.degree()==0]
g.delete_vertices(todelete)
todelete = [edge for edge in g.es if edge["weight"]>4]
g.delete_edges(todelete)

In [545]:
clustering = g.community_multilevel(weights="weight")

In [590]:
x = [(0,10),
 (2,12),
 (4,14),
 (6,16),
 (8,18),
 (10,20),
 (12,22),
 (14,24),
 (16,26),
 (18,28),
 (20,30),
 (22,32)]

In [ ]:
CompletePeriods = ExtractCompletePeriods(x,tolerancevalue)